# Text Mining  

El siguiente notebook realiza una extracccion de las informes estenográficos de AMLO y la presidenta actual de Mexico Clausia Sheimbaum. 

Eric Lemus Avalos

In [17]:
# Librerias 
import os
import wget
import glob
import re 
import time 
import nltk 
import requests
from datetime import datetime

# Pulling data out HTML file
from bs4 import BeautifulSoup

# Manipulación de Data 
import polars as pl 

In [2]:
workdir = os.getcwd()
workdir

'C:\\Users\\ericl\\tareas_nlp'

### Versiones estenograficas de AMLO 

In [9]:
paginas_dir = os.path.join(workdir, "paginas")

if not os.path.exists(paginas_dir):
    os.makedirs(paginas_dir)

url_pagina_i = 'https://amlo.presidente.gob.mx/secciones/version-estenografica/page/'
num_pag = 153 # paginas validad con versiones estenograficas
for i in range(num_pag):
    wget.download(url= url_pagina_i + str(i+1), out = "./paginas/" + str(i+1) + ".txt") 

Buscamos el patron de los urls de las versiones estenográficas para Andrés Manuel López Obrador. 

In [12]:
paginas_path = r'./paginas/*'
pattern = r'href="(https://amlo\.presidente\.gob\.mx/\d{2}-\d{2}-\d{2}-version-estenografica-[^"]+)"'

In [13]:
# Lista para almacenar las URLs encontradas
urls_config = []

# Procesar los archivos en la carpeta
for file_pagina in glob.glob(paginas_path):
    # print(f"Procesando archivo: {file_pagina}")
    with open(file_pagina, 'r', encoding='utf-8') as f:
        content = f.read()     
        # Buscar todas las coincidencias en el contenido del archivo
        urls_encontradas = re.findall(pattern, content) 
        # Agregar las URLs encontradas a la lista
        urls_config.extend(urls_encontradas)

In [14]:
urls_config[:10]

['https://amlo.presidente.gob.mx/30-09-24-version-estenografica-de-la-ultima-conferencia-de-prensa-del-presidente-andres-manuel-lopez-obrador/',
 'https://amlo.presidente.gob.mx/30-09-24-version-estenografica-de-la-ultima-conferencia-de-prensa-del-presidente-andres-manuel-lopez-obrador/',
 'https://amlo.presidente.gob.mx/29-09-24-version-estenografica-tren-maya-y-entrega-de-reconocimientos-a-ingenieros-constructores-desde-chetumal-quintana-roo/',
 'https://amlo.presidente.gob.mx/29-09-24-version-estenografica-tren-maya-y-entrega-de-reconocimientos-a-ingenieros-constructores-desde-chetumal-quintana-roo/',
 'https://amlo.presidente.gob.mx/28-09-24-version-estenografica-inauguracion-zona-de-riego-de-la-presa-santa-maria-desde-el-rosario-sinaloa/',
 'https://amlo.presidente.gob.mx/28-09-24-version-estenografica-inauguracion-zona-de-riego-de-la-presa-santa-maria-desde-el-rosario-sinaloa/',
 'https://amlo.presidente.gob.mx/27-09-24-version-estenografica-entrega-de-acueducto-y-distrito-de-rie

In [15]:
# Numero de versiones estenograficas encontradas
len(set(urls_config))

1963

In [16]:
urls = list(set(urls_config))
urls[:5]

['https://amlo.presidente.gob.mx/26-09-19-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador/',
 'https://amlo.presidente.gob.mx/13-12-23-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador/',
 'https://amlo.presidente.gob.mx/02-08-24-version-estenografica-inauguracion-del-puente-la-concordia-desde-chiapas/',
 'https://amlo.presidente.gob.mx/13-12-19-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador/',
 'https://amlo.presidente.gob.mx/06-10-19-version-estenografica-dialogo-con-la-comunidad-del-hospital-rural-huajuapan-de-leon-oaxaca/']

Guardamos los urls de las versiones estenogáficas

In [17]:
with open('conferencias_urls.txt', 'w', encoding='utf-8') as file:
    for url in urls:
        file.write(url + "\n")

Utilizamos ```requests``` 

Parseamos del HTML para obtener el texto limpio. 

In [18]:
import requests

#ejemplo de url
url = "https://amlo.presidente.gob.mx/21-07-20-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador/"

response = requests.get(url)
if response.status_code == 200:
    # Parsear el contenido HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar cintenido 
    contenido_relevante = soup.find("div", class_="entry-content")
    
    if contenido_relevante:
        texto = contenido_relevante.get_text(strip=True)
        # print(texto)
    else:
        print("No se encontró contenido relevante en la página.")

else:
    print(f"Error al acceder a la página: {response.status_code}")



In [15]:
texto[:1000] #mostramps los primeros 1000 caracteres. 

'2020, Año de Leona Vicario, Benemérita Madre de la PatriaPRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR:Buenos días.Vamos a tratar el día de hoy, como todos los martes, el tema de la salud, de cómo se va a avanzando en el combate a la pandemia. Nos van a informar, tanto el doctor Alcocer como el doctor Hugo López-Gatell.También va a informar Marcelo Ebrard sobre la situación de nuestros paisanos en el extranjero y los protocolos, el avance, la participación de México para la creación y distribución, obtención de la vacuna. Eso también es un tema importante.Tenemos que seguir enfrentando esta pandemia, que va a la baja; muy lentamente, pero a baja. No hemos tenido problemas mayores en cuanto a hospitalización, no ha habido saturación de hospitales, se está atendiendo diariamente esta demanda; donde se requiere, todos juntos, todos los integrantes del sector salud actúan para que haya camas, para que haya equipos, para que no falten los especialistas y vamos avanzando, saliendo adelante frente 

#### Extracción del texto para cada versión estenografica

In [19]:
output_folder = "versiones_estenograficas_amlo"
os.makedirs(output_folder, exist_ok=True)

In [20]:

with open('conferencias_urls.txt', 'r', encoding='utf-8') as file:
    urls = [line.strip() for line in file]

fallidos = []

for url in urls:
    print(f"Procesando: {url}")
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            # Parsear el contenido HTML de la pagina
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Buscar el contenido inspeccionado
            contenido_relevante = soup.find("div", class_="entry-content")
            
            if contenido_relevante:
                texto = contenido_relevante.get_text(strip=True)
                nombre_archivo = url.split("/")[-2] 
                archivo_salida = os.path.join(output_folder, f"{nombre_archivo}.txt")
                
                with open(archivo_salida, 'w', encoding='utf-8') as output_file:
                    output_file.write(texto)
            else:
                # print(f"No se encontró contenido relevante en {url}")
                fallidos.append(url)
        else:
            # print(f"Error al acceder a {url}: {response.status_code}")
            fallidos.append(url)
    except Exception as e:
        print(f"Error procesando {url}: {e}")
        fallidos.append(url)

if fallidos:
    print(f"\nNúmero de conferencias que no se pudieron descargar: {len(fallidos)}")
    print("URLs de las conferencias fallidas:")
    for fallido in fallidos:
        print(fallido)


Procesando: https://amlo.presidente.gob.mx/22-08-23-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador/
Procesando: https://amlo.presidente.gob.mx/08-05-24-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador/
Procesando: https://amlo.presidente.gob.mx/12-10-20-version-estenografica-graduacion-15a-generacion-del-curso-de-formacion-del-servicio-de-proteccion-federal/
Procesando: https://amlo.presidente.gob.mx/21-02-20-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-desde-la-paz-baja-california-sur/
Procesando: https://amlo.presidente.gob.mx/13-10-21-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador/
Procesando: https://amlo.presidente.gob.mx/28-02-22-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador/
Procesando: https://amlo.presiden

### Versiones estenografica de Claudia Sheinbaum Pardo

In [2]:
url_pagina_i = 'https://www.gob.mx/presidencia/es/archivo/articulos?idiom=es&order=DESC&page='
num_pag = 16 #hasta la fecha actual 27-enero-2025, solo hay 16 paginas 

workdir = os.getcwd()
paginas_dir = os.path.join(workdir, "paginas_sheinbaum")
if not os.path.exists(paginas_dir):
    os.makedirs(paginas_dir)

for i in range(num_pag):
    wget.download(url= url_pagina_i + str(i+1), out = "./paginas_sheinbaum/" + str(i+1) + ".txt") 


In [7]:
for file_pagina in glob.glob('./paginas_sheinbaum/*'):
    with open(file_pagina, 'r', encoding='utf-8') as f:
        content = f.read()
        print(f"Contenido de {file_pagina}:\n{content[:500]}\n")

Contenido de ./paginas_sheinbaum\1.txt:
<!DOCTYPE html>
<html class='no-js' lang='es'>
<head>
    <title>
  Blog | Archivo | Presidencia de la República  | Gobierno | gob.mx
</title>

  <meta charset='UTF-8'>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <meta name="theme-color" content="#393C3E">

  
  <meta content="Archivo Blog" property="og:title" />
  <meta name="description" content="Archivo de artículos de Presidencia de la República ">


Contenido de ./paginas_sheinbaum\10.txt:
<!DOCTYPE html>
<html class='no-js' lang='es'>
<head>
    <title>
  Blog | Archivo | Presidencia de la República  | Gobierno | gob.mx
</title>

  <meta charset='UTF-8'>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <meta name="theme-color" content="#393C3E">

  
  <meta content="Archivo Blog" property="og:title" />
  <meta 

#### Extrar urls

In [14]:
def extraer_urls(ruta_archivo):
    """Extrae e imprime todos los URLs presentes en un archivo de texto."""
    pattern = r'https?://[^\s"]+'
    
    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        contenido = f.read()
        urls = re.findall(pattern, contenido)
    
    if urls:
        print("\n".join(urls))
    else:
        print(f"No se encontraron URLs en {ruta_archivo}")

ruta_archivo = os.path.join(workdir, "paginas_sheinbaum", "1.txt")
extraer_urls(ruta_archivo)

https://fonts.googleapis.com/css?family=Open+Sans:400,700,600,300'
https://framework-gb.cdn.gob.mx/applications/cms/favicon.png'
https://framework-gb.cdn.gob.mx/gm/accesibilidad/css/gobmx-accesibilidad.min.css
https://oss.maxcdn.com/libs/html5shiv/3.7.0/html5shiv.js
https://oss.maxcdn.com/libs/respond.js/1.4.2/respond.min.js
https://www.googletagmanager.com/gtag/js?id=G-M3KVJ03TMZ
https://framework-gb.cdn.gob.mx/applications/cms/carrusel/swiper-bundle.min.js
https://framework-gb.cdn.gob.mx/applications/cms/carrusel/swiper-bundle.min.css
https://cdnjs.cloudflare.com/ajax/libs/slick-carousel/1.6.0/slick.js
https://s.go-mpulse.net/boomerang/
https://s2.go-mpulse.net/boomerang/
https://connect.facebook.net/es_LA/sdk.js#xfbml=1&version=v13.0
https://www.gob.mx/busqueda?utf8=✓
https://framework-gb.cdn.gob.mx/landing/img/lupa.png
https://transparencia.presidencia.gob.mx/presidencia/transparencia/politicas-de-tratamiento-de-datos-personales-de-la-oficina-de-la-presidencia-de-la-republica/'>
ht

Los urls de las conferencias estan asociados cuando ocurre el evento (Botón) "Continuar leyendo"

In [16]:
def extraer_urls_versiones_estenograficas(archivo):
    """Extrae e imprime los URLs completos de las versiones estenográficas desde un archivo HTML."""
    workdir = os.getcwd()
    ruta_archivo = os.path.join(workdir, archivo)

    with open(ruta_archivo, 'r', encoding='utf-8') as f:
        contenido = f.read()

    # Parsear el HTML con BeautifulSoup
    soup = BeautifulSoup(contenido, 'html.parser')

    # Buscar todos los enlaces que contienen el texto "Continuar Leyendo"
    enlaces = soup.find_all('a', string=re.compile("Continuar Leyendo", re.IGNORECASE))

    base_url = "https://www.gob.mx"

    # Construir urls 
    urls = [
        enlace['href'] if enlace['href'].startswith("http") else base_url + enlace['href']
        for enlace in enlaces if enlace.has_attr('href')]

    if urls:
        print("\n".join(urls))
    else:
        print(f"No se encontraron URLs en {ruta_archivo}")

# Ejemplo (insight)
extraer_urls_versiones_estenograficas("paginas_sheinbaum/2.txt")



https://www.gob.mx/presidencia/es/articulos/version-estenografica-entrega-de-tarjetas-de-las-pensiones-para-el-bienestar?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-recorrido-por-el-centro-de-rehabilitacion-e-inclusion-teleton-crit-de-tlapa-de-comonfort-guerrero?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-arranque-del-programa-nacional-de-fertilizantes-para-el-bienestar?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-entrega-de-tarjetas-de-la-pension-mujeres-bienestar-388297?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-entrega-de-tarjetas-jovenes-construyendo-el-futuro?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-de-la-presidenta-claudia-sheinbaum-pardo-del-17-de-enero-de-2025?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-centro-integral-para-el-programa-acapulco-se-transforma-contigo?idio

In [18]:
def extraer_urls_versiones_estenograficas(archivo):
    """Extrae los URLs completos de las versiones estenográficas desde un archivo HTML."""
    with open(archivo, 'r', encoding='utf-8') as f:
        contenido = f.read()

    # Parsear el HTML con BeautifulSoup
    soup = BeautifulSoup(contenido, 'html.parser')

    # Buscar todos los enlaces que contienen el texto "Continuar Leyendo"
    enlaces = soup.find_all('a', string=re.compile("Continuar Leyendo", re.IGNORECASE))

    base_url = "https://www.gob.mx"

    # Construir urls
    urls = [
        enlace['href'] if enlace['href'].startswith("http") else base_url + enlace['href']
        for enlace in enlaces if enlace.has_attr('href')
    ]

    return urls

num_pag = 16
workdir = os.getcwd()
paginas_dir = os.path.join(workdir, "paginas_sheinbaum")

archivos_txt = [os.path.join(paginas_dir, f) for f in os.listdir(paginas_dir) if f.endswith('.txt')]

todos_los_urls = []
for archivo in archivos_txt:
    urls_encontradas = extraer_urls_versiones_estenograficas(archivo)
    if urls_encontradas:
        print(f"URLs encontradas en {archivo}:")
        print("\n".join(urls_encontradas))
        todos_los_urls.extend(urls_encontradas)

# Guardar urls en un txt 
output_file = os.path.join(workdir, "conferencias_urls_sheinbaum.txt")
with open(output_file, "w", encoding="utf-8") as f:
    for url in todos_los_urls:
        f.write(url + "\n")

URLs encontradas en C:\Users\ericl\tareas_nlp\paginas_sheinbaum\1.txt:
https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-de-la-presidenta-claudia-sheinbaum-pardo-del-27-de-enero-de-2025?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-entrega-de-tarjetas-pension-mujeres-bienestar?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-programas-para-el-bienestar-388896?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-inauguracion-de-la-carretera-mitla-tehuantepec?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-de-la-presidenta-claudia-sheinbaum-pardo-del-24-de-enero-de-2025?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-de-la-presidenta-claudia-sheinbaum-pardo-del-23-de-enero-de-2025?idiom=es
https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-de

In [31]:
%pip install locate 

Note: you may need to restart the kernel to use updated packages.


In [36]:
import locale  
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

output_folder = "versiones_estenograficas_sheinbaum"
os.makedirs(output_folder, exist_ok=True)

with open('conferencias_urls_sheinbaum.txt', 'r', encoding='utf-8') as file:
    urls = [line.strip() for line in file]

print(f'Conferencias encontradas: {len(set(urls))}')

Conferencias encontradas: 143


#### Extraer texto relevante por conferencia

In [29]:
def extraer_fecha(soup):
    """
    Extrae la fecha del contenido HTML.
    La fecha está en un elemento <p> que contiene "Presidencia de la República".
    """
    # Buscar el elemento <p> que contiene "Presidencia de la República"
    fecha_element = soup.find("p", string=lambda x: x and "Presidencia de la República" in x)
    
    if fecha_element:
        fecha_texto = fecha_element.get_text(strip=True)
        fecha = fecha_texto.split("|")[-1].strip()  # Extraer la fecha después del carácter "|"
        try:
            # Convertir la fecha a formato estándar (YYYY-MM-DD)
            fecha_formateada = datetime.strptime(fecha, "%d de %B de %Y").strftime("%Y-%m-%d")
            return fecha_formateada
        except ValueError as e:
            print(f"Error al convertir la fecha '{fecha}': {e}")
    return None

In [34]:
%%time
fallidos = []

for url in urls:
    print(f"Procesando: {url}")
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            # Parsear el contenido HTML de la página
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Buscar el contenido relevante
            contenido_relevante = soup.find("div", class_="article-body")
            fecha = extraer_fecha(soup)
            
            if contenido_relevante and fecha:
                texto = contenido_relevante.get_text(strip=True)
                
                # Extraer el nombre del archivo del URL y agregar la fecha
                nombre_archivo = url.split("/articulos/")[-1].split("?")[0]
                nombre_archivo_con_fecha = f"{fecha}_{nombre_archivo}"
                archivo_salida = os.path.join(output_folder, f"{nombre_archivo_con_fecha}.txt")
                
                # Guardar el contenido en un archivo
                with open(archivo_salida, 'w', encoding='utf-8') as output_file:
                    output_file.write(texto)
                print(f"Archivo guardado: {archivo_salida}")
            else:
                print(f"No se encontró contenido relevante o fecha en {url}")
                fallidos.append(url)
        else:
            print(f"Error al acceder a {url}: {response.status_code}")
            fallidos.append(url)
    except Exception as e:
        print(f"Error procesando {url}: {e}")
        fallidos.append(url)

Procesando: https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-de-la-presidenta-claudia-sheinbaum-pardo-del-27-de-enero-de-2025?idiom=es
Archivo guardado: versiones_estenograficas_sheinbaum_2\2025-01-27_version-estenografica-conferencia-de-prensa-de-la-presidenta-claudia-sheinbaum-pardo-del-27-de-enero-de-2025.txt
Procesando: https://www.gob.mx/presidencia/es/articulos/version-estenografica-entrega-de-tarjetas-pension-mujeres-bienestar?idiom=es
Archivo guardado: versiones_estenograficas_sheinbaum_2\2025-01-25_version-estenografica-entrega-de-tarjetas-pension-mujeres-bienestar.txt
Procesando: https://www.gob.mx/presidencia/es/articulos/version-estenografica-programas-para-el-bienestar-388896?idiom=es
Archivo guardado: versiones_estenograficas_sheinbaum_2\2025-01-25_version-estenografica-programas-para-el-bienestar-388896.txt
Procesando: https://www.gob.mx/presidencia/es/articulos/version-estenografica-inauguracion-de-la-carretera-mitla-tehuantepec?id

In [35]:
if fallidos:
    print(f"\nNúmero de conferencias que no se pudieron descargar: {len(fallidos)}")
    print("URLs de las conferencias fallidas:")
    for fallido in fallidos:
        print(fallido)
else:
    print("Todas las conferencias se procesaron correctamente.")

Todas las conferencias se procesaron correctamente.


### Preproceso de la data

Cambiaremos el nombre a los archivos ue tienen mismas fechas y los guardamos en un dataframe de polars. 

In [189]:
# Extraer fechas amlo
def extract_date_from_filename(filename):
    # Extraer la fecha del nombre del archivo (formato: dd-mm-aa-version-estenografica-...)
    date_str = filename.split("-")[:3]  # Obtener los primeros tres elementos (dd, mm, aa)
    date_str = "-".join(date_str)  # Unirlos en formato dd-mm-aa
    try:
        # Convertir a formato de fecha estándar (yyyy-mm-dd)
        date_obj = datetime.strptime(date_str, "%d-%m-%y")
        return date_obj.strftime("%Y-%m-%d")  # Devolver en formato yyyy-mm-dd
    except ValueError:
        return None  

def get_title(filename):
    return " ".join(filename.split("-")[3:]).replace(".txt", "").capitalize()

In [190]:
%%time
folder_path = "versiones_estenograficas_amlo"  

titles = []
contents = []
authors = []
dates = []

for file in os.listdir(folder_path):
    if file.endswith(".txt"):
        file_path = os.path.join(folder_path, file)
        
        # Leer el contenido 
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        date = extract_date_from_filename(file)
        
        titles.append(get_title(file))
        contents.append(content)
        authors.append("AMLO")
        dates.append(date)

df_amlo = pl.DataFrame({
    "Conferencia": titles,
    "Contenido": contents,
    "Autor": authors,
    "Fecha": dates
})


CPU times: total: 156 ms
Wall time: 601 ms


In [191]:
print(df_amlo.head())
print(df_amlo.tail())

shape: (5, 4)
┌─────────────────────────────────┬─────────────────────────────────┬───────┬────────────┐
│ Conferencia                     ┆ Contenido                       ┆ Autor ┆ Fecha      │
│ ---                             ┆ ---                             ┆ ---   ┆ ---        │
│ str                             ┆ str                             ┆ str   ┆ str        │
╞═════════════════════════════════╪═════════════════════════════════╪═══════╪════════════╡
│ Version estenografica programa… ┆ 2020, Año de Leona Vicario, Be… ┆ AMLO  ┆ 2020-02-01 │
│ Version estenografica programa… ┆ 2020, Año de Leona Vicario, Be… ┆ AMLO  ┆ 2020-02-01 │
│ Version estenografica de la co… ┆ 2022: Año de Ricardo Flores Ma… ┆ AMLO  ┆ 2022-02-01 │
│ Version estenografica de la co… ┆ 2023: Año de Francisco Villa, … ┆ AMLO  ┆ 2023-02-01 │
│ Version estenografica de la co… ┆ 2021: Año de la IndependenciaP… ┆ AMLO  ┆ 2021-03-01 │
└─────────────────────────────────┴─────────────────────────────────┴───────

In [192]:
df_amlo_sorted = df_amlo.sort("Fecha")
print(df_amlo_sorted.head())
print(df_amlo_sorted.tail())

shape: (5, 4)
┌─────────────────────────────────┬─────────────────────────────────┬───────┬────────────┐
│ Conferencia                     ┆ Contenido                       ┆ Autor ┆ Fecha      │
│ ---                             ┆ ---                             ┆ ---   ┆ ---        │
│ str                             ┆ str                             ┆ str   ┆ str        │
╞═════════════════════════════════╪═════════════════════════════════╪═══════╪════════════╡
│ Version estenografica inaugura… ┆ 2024: Año de Felipe Carrillo P… ┆ AMLO  ┆ null       │
│ Version estenografica dialogo … ┆ 2019, Año del Caudillo del Sur… ┆ AMLO  ┆ 2009-10-11 │
│ Version estenografica de la co… ┆ 2019, Año del Caudillo del Sur… ┆ AMLO  ┆ 2016-12-17 │
│ Version estenografica programa… ┆ 2019, Año del Caudillo del Sur… ┆ AMLO  ┆ 2019-04-13 │
│ Version estenografica de la co… ┆ Mañana vamos a Pachuca, Hidalg… ┆ AMLO  ┆ 2019-05-07 │
└─────────────────────────────────┴─────────────────────────────────┴───────

Obtenemos las conferencias que no tiene fechas validas y las corregimos manualmente. 

In [193]:
df_invalid_dates = df_amlo.filter(
    (pl.col("Fecha").is_null()) | 
    (~pl.col("Fecha").str.slice(0, 4).is_in(["2019", "2020", "2021", "2022", "2023", "2024"])))
print(df_invalid_dates)

shape: (4, 4)
┌─────────────────────────────────┬─────────────────────────────────┬───────┬────────────┐
│ Conferencia                     ┆ Contenido                       ┆ Autor ┆ Fecha      │
│ ---                             ┆ ---                             ┆ ---   ┆ ---        │
│ str                             ┆ str                             ┆ str   ┆ str        │
╞═════════════════════════════════╪═════════════════════════════════╪═══════╪════════════╡
│ Version estenografica conferen… ┆ 2020, Año de Leona Vicario, Be… ┆ AMLO  ┆ 2030-03-03 │
│ Version estenografica dialogo … ┆ 2019, Año del Caudillo del Sur… ┆ AMLO  ┆ 2009-10-11 │
│ Version estenografica de la co… ┆ 2019, Año del Caudillo del Sur… ┆ AMLO  ┆ 2016-12-17 │
│ Version estenografica inaugura… ┆ 2024: Año de Felipe Carrillo P… ┆ AMLO  ┆ null       │
└─────────────────────────────────┴─────────────────────────────────┴───────┴────────────┘


Para el elemento `df_invalid_dates[0, :]` no es incluido en el análisis.

Las fechas de los otros tres elementos se corrigieron manualmente de la siguiente manera:

- null → **28 de junio de 2024**
- 2009-10-11 → **11 de octubre de 2019**
- 2016-12-17 → **17 diciembre de 2019**


In [194]:
df_amlo = df_amlo.with_columns(
    pl.when(pl.col("Fecha").is_null()).then(pl.lit("2024-06-28"))
    .when(pl.col("Fecha") == pl.lit("2009-10-11")).then(pl.lit("2019-10-11"))
    .when(pl.col("Fecha") == pl.lit("2016-12-17")).then(pl.lit("2019-12-17"))
    .otherwise(pl.col("Fecha"))
    .alias("Fecha")
)


In [195]:
# Eliminar elemento asociado con 2030-03-03
df_amlo = df_amlo.filter(pl.col("Fecha") != "2030-03-03")

Ordenamos las conferencias por fecha. Si existen fechas repetidas, añadimos una nueva columna llamada 'Repeticion' 
para asignar un número consecutivo a cada ocurrencia de la misma fecha.

In [196]:
df_amlo = df_amlo.with_columns(pl.struct(["Fecha"]).rank("ordinal").over("Fecha").alias("Repeticion"))
df_amlo = df_amlo.sort("Fecha")

In [197]:
print(df_amlo.head())
print(df_amlo.tail())

shape: (5, 5)
┌───────────────────────────────────┬────────────────────────────┬───────┬────────────┬────────────┐
│ Conferencia                       ┆ Contenido                  ┆ Autor ┆ Fecha      ┆ Repeticion │
│ ---                               ┆ ---                        ┆ ---   ┆ ---        ┆ ---        │
│ str                               ┆ str                        ┆ str   ┆ str        ┆ u32        │
╞═══════════════════════════════════╪════════════════════════════╪═══════╪════════════╪════════════╡
│ Version estenografica programa…   ┆ 2019, Año del Caudillo del ┆ AMLO  ┆ 2019-04-13 ┆ 1          │
│                                   ┆ Sur…                       ┆       ┆            ┆            │
│ Version estenografica de la co…   ┆ Mañana vamos a Pachuca,    ┆ AMLO  ┆ 2019-05-07 ┆ 1          │
│                                   ┆ Hidalg…                    ┆       ┆            ┆            │
│ Version estenografica de la co…   ┆ 2019, Año del Caudillo del ┆ AMLO  ┆ 20

In [203]:
print(f"Número de conferecnias: {df_amlo.height}")
print(f"Número de columnas: {df_amlo.width}")
print(f"Columnas: {df_amlo.columns}")
print(f"Shape: {df_amlo.shape}")

Número de conferecnias: 1961
Número de columnas: 5
Columnas: ['Conferencia', 'Contenido', 'Autor', 'Fecha', 'Repeticion']
Shape: (1961, 5)


**Ahora para Sheinbaum** 

In [177]:
def extract_date_from_filename(filename):
    date_str = filename.split("_")[0]  # Obtener la parte antes del primer guión bajo (yyyy-mm-dd)
    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")
        return date_obj.strftime("%Y-%m-%d")  
    except ValueError:
        return None

def get_title(filename):
    return " ".join(filename.split("-")[3:]).replace(".txt", "").capitalize()


In [209]:
%%time
folder_path = "versiones_estenograficas_sheinbaum"  
titles = []
contents = []
authors = []
dates = []

for file in os.listdir(folder_path):
    if file.endswith(".txt"):
        file_path = os.path.join(folder_path, file)
        
        # Leer el contenido 
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        date = extract_date_from_filename(file)
        
        titles.append(get_title(file))
        contents.append(content)
        authors.append("Sheinbaum")
        dates.append(date)

df_shein = pl.DataFrame({
    "Conferencia": titles,
    "Contenido": contents,
    "Autor": authors,
    "Fecha": dates
})

CPU times: total: 15.6 ms
Wall time: 46.9 ms


In [183]:
df_shein_sorted = df_shein.sort("Fecha")
print(df_shein_sorted.head())
print(df_shein_sorted.tail())

shape: (5, 4)
┌─────────────────────────────────┬─────────────────────────────────┬───────────┬────────────┐
│ Conferencia                     ┆ Contenido                       ┆ Autor     ┆ Fecha      │
│ ---                             ┆ ---                             ┆ ---       ┆ ---        │
│ str                             ┆ str                             ┆ str       ┆ str        │
╞═════════════════════════════════╪═════════════════════════════════╪═══════════╪════════════╡
│ Estenografica mensaje de la pr… ┆ PRESIDENTA CLAUDIA SHEINBAUM P… ┆ Sheinbaum ┆ 2024-10-01 │
│ Estenografica toma de protesta… ┆ MODERADORA 1: Mexicanas y mexi… ┆ Sheinbaum ┆ 2024-10-01 │
│ Estenografica toma de protesta… ┆ IFIGENIA MARTÍNEZ Y HERNÁNDEZ,… ┆ Sheinbaum ┆ 2024-10-01 │
│ Estenografica conferencia de p… ┆ PRESIDENTA CLAUDIA SHEINBAUM P… ┆ Sheinbaum ┆ 2024-10-02 │
│ Estenografica conferencia de p… ┆ PRESIDENTA CLAUDIA SHEINBAUM P… ┆ Sheinbaum ┆ 2024-10-03 │
└─────────────────────────────────┴─

In [184]:
df_shein = df_shein.with_columns(pl.struct(["Fecha"]).rank("ordinal").over("Fecha").alias("Repeticion"))
df_shein = df_shein.sort("Fecha")

In [186]:
print(df_shein.head(10))

shape: (10, 5)
┌──────────────────────────────┬─────────────────────────────┬───────────┬────────────┬────────────┐
│ Conferencia                  ┆ Contenido                   ┆ Autor     ┆ Fecha      ┆ Repeticion │
│ ---                          ┆ ---                         ┆ ---       ┆ ---        ┆ ---        │
│ str                          ┆ str                         ┆ str       ┆ str        ┆ u32        │
╞══════════════════════════════╪═════════════════════════════╪═══════════╪════════════╪════════════╡
│ Estenografica mensaje de la  ┆ PRESIDENTA CLAUDIA          ┆ Sheinbaum ┆ 2024-10-01 ┆ 1          │
│ pr…                          ┆ SHEINBAUM P…                ┆           ┆            ┆            │
│ Estenografica toma de        ┆ MODERADORA 1: Mexicanas y   ┆ Sheinbaum ┆ 2024-10-01 ┆ 2          │
│ protesta…                    ┆ mexi…                       ┆           ┆            ┆            │
│ Estenografica toma de        ┆ IFIGENIA MARTÍNEZ Y         ┆ Sheinbaum ┆ 2

In [202]:
print(f"Número de conferecnias: {df_shein.height}")
print(f"Número de columnas: {df_shein.width}")
print(f"Columnas: {df_shein.columns}")
print(f"Shape: {df_shein.shape}")

Número de conferecnias: 143
Número de columnas: 5
Columnas: ['Conferencia', 'Contenido', 'Autor', 'Fecha', 'Repeticion']
Shape: (143, 5)


### Guardar conferencias

In [207]:
# Combinamos los dataframes
df_combined = pl.concat([df_shein, df_amlo], how="vertical")

In [ ]:
if not os.path.exists("data"):
    os.makedirs("data")

df.write_parquet("data/transcripts.parquet")
df.write_csv("data/transcripts.csv")


In [208]:
%%time
if not os.path.exists("data"):
    os.makedirs("data")

df_combined.write_parquet("data/transcripts.parquet")
df_combined.write_csv("data/transcripts.csv")

CPU times: total: 531 ms
Wall time: 696 ms
